In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
class movie(object):

    movie_code = ''
    
    
    def __init__(self,name):
        self.name = name
        
    def find_movie_code(self):
        movie.movie_code = ''
        search_url = 'https://search.naver.com/search.naver'
        query = self.name
        params = {
            'where': 'nexearch',
            'query':query
        }
        search_resp = requests.get(search_url,params = params)
        movie_code = ''
        search_soup = BeautifulSoup(search_resp.content)
        i = 99
        try:
            search_movie_url = search_soup.find('div',class_='info_main').find('h3').a['href']
            for idx,val in enumerate(search_movie_url):
                if val == '=':
                    i = idx
                if idx > i:
                    movie.movie_code += val
            print(movie.movie_code)
            return movie.movie_code
        
        
        
        except:
            print('영화제목을 똑바로 쳐주세요!')
            
            
            
    def find_story(self):
        new_dict = {}
        new_list = []
        movie_story_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
        params = {
            'code' : movie.movie_code
        }
        resp = requests.get(movie_story_url, params)
        soup = BeautifulSoup(resp.content,'html.parser')
        new_dict['movie_story_title'] = soup.find('div',class_='story_area').find('h5',class_='h_tx_story').text
        new_dict['movie_story_content'] = soup.find('div',class_='story_area').find('p',class_='con_tx').text
        
        new_list.append(new_dict)
        print('find_story_loading_complete')
        return new_list
        
        
        
    def find_best_lines(self):
        movie_best_url = 'https://movie.naver.com/movie/bi/mi/script.nhn'
        i = 1
        new_list = []
        while True:
            params = {
            'code' : movie.movie_code,
            'page': i
            }
            resp = requests.get(movie_best_url,params)
            soup = BeautifulSoup(resp.content,'html.parser')
            ul_tag = soup.find('ul',class_ = 'lines')
            if i%50 ==0:
                print(f'{i}번째 페이지 로딩중')
            cnt = 0
            if soup.find('div',class_='paging').find('a',title = '다음'):
                for ul in ul_tag.contents:
                    new_dict= {}
                    if type(ul) == type(ul_tag.contents[0]):
                        continue
        #             print(ul)
                    new_dict['best_lines'] = ul.find('div').find('p',class_='one_line').text
                    if not ul.find('div').find('p',class_='char_part')==None:
                        new_dict['cast'] = ul.find('div').find('p',class_='char_part').text
                        new_dict['actor'] = ul.find('div').find('p',class_='char_part').a.text
                        new_dict['line_desc'] = ul.find('div').find('p',class_='line_desc').text
                        new_list.append(new_dict)
                i += 1
            else:
                break
#         print(len(new_list))
        print('find_best_lines_loading_complete')
        df_list = pd.DataFrame(new_list)
        
        
        return df_list

In [5]:
movie1 = movie('설국열차')


In [6]:
movie_code = movie1.find_movie_code()
movie_best_lines = movie1.find_best_lines()
movie_story = movie1.find_story()

62328
50번째 페이지 로딩중
100번째 페이지 로딩중
150번째 페이지 로딩중
200번째 페이지 로딩중
find_best_lines_loading_complete
find_story_loading_complete


In [7]:
df_movie_best_lines = pd.DataFrame(movie_best_lines)

In [8]:
df_movie_best_lines.to_csv('영화리뷰.csv',encoding = 'cp949')